In [1]:
import random, shutil, os
import numpy as np
import pandas as pd

from mido import MidiFile
from glob import glob

In [ ]:
def create_only_folders(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir,f))]


shutil.copytree('/Users/cooky/Desktop/groove', './milisec_dataset', ignore=create_only_folders)

In [2]:
midi_list = glob('/Users/cooky/Desktop/groove/**/**/*.mid')

In [99]:
beat_list = glob('./milisec_200/beat*.gzip')
fill_list = glob('./milisec_200/fill*.gzip')

In [100]:
minb = 9999
maxb = -1

for l in beat_list:
    a = pd.read_csv(l, compression='gzip')
    if minb > a.shape[0]:
        minb = a.shape[0]
    if maxb < a.shape[0]:
        maxb = a.shape[0]

minf = 9999
maxf = -1

for l in fill_list:
    a = pd.read_csv(l, compression='gzip')
    if minf > a.shape[0]:
        minf = a.shape[0]
    if maxf < a.shape[0]:
        maxf = a.shape[0]

In [101]:
print(minb, maxb, minf, maxf)

200 21599 203 2255


In [38]:
def kit_classify(note):
    cat = ''
    if note in [36]: # kick
        cat = 0
    elif note in [38, 40, 37]: # snare 
        cat = 1
    elif note in [48, 50]: # High Tom
        # cat = 2
        cat = 2 # Replace HT -> MT
    elif note in [45, 47]: # Mid Tom
        # cat = 3
        cat = 2
    elif note in [43, 58]: # Low Tom
        # cat = 4
        cat = 3
    elif note in [46, 26]: # Open Hat
        # cat = 5
        cat = 4
    elif note in [42, 22, 44]: # Closed Hat
        # cat = 6
        cat = 5
    elif note in [49, 55, 57, 52]: # Crash
        # cat = 7 
        cat = 6 # Replace Cr -> Ri
    elif note in [51, 59, 53]: # Ride
        cat = 6
    else:
        print(note)

    return cat

In [2]:

seven_kind_arr = [
   # bd sn mt lt oh ch ri 
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0],
    [1, 1, 0, 0, 1, 0, 0],
    [1, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0]
]



In [97]:
# mid2 = MidiFile('/Users/cooky/Desktop/groove/drummer7/session1/3_rock_86_fill_4-4.mid', clip=True)
from turtle import onscreenclick


beat_count = 1
fill_count = 1
max = -1
min = 9999

for midi in midi_list:
    mid = MidiFile(midi, clip=True)
    on_sec = 0
    event = []
    first_event = -1

    for msg in mid.tracks[0]:
            note = str(msg).split()
            if len(note) == 5 and note[0][0] != '<':
                on_sec += msg.dict()['time']

                # if on_sec > 26600 and on_sec < 26999:
                #         print(msg)

                if note[0] == 'note_on':
                    if first_event < 0:
                        first_event = on_sec
                    
                    if [round((on_sec - first_event)/10), msg.dict()['note']] not in event:
                        event.append([round((on_sec - first_event)/10), msg.dict()['note']])
                        # print(on_sec)
                        # print(msg.dict()['note'])
    # print(len(event))
    # print(event)
    # print('---')
    # if count > 2:
    #     break
    # print(event[-1][0])

    beat = pd.DataFrame(0, columns=np.arange(7), index=np.arange(event[-1][0] + 1))

    for e in event:
        beat.loc[e[0],kit_classify(e[1])] = 1

    # print(beat)

    softmax_beat = []
    # print(midi)
    # print(beat.to_numpy().tolist().index([0, 1, 0, 0, 1, 1, 0]))
    # print(beat.to_string())

    not_in = []

    for i in range(len(beat)):
        if beat.to_numpy()[i].tolist() not in seven_kind_arr:
            not_in.append(i)

    # print(not_in)

    filestr = ''

    
       

    if len(not_in) != 0:

        for i in range(len(not_in)+1):

            if 'beat' in midi.lower():
                filestr = 'beat' + str(beat_count)
            elif 'fill' in midi.lower():
                filestr = 'fill' + str(fill_count)

            if i == 0:
                imsi_beat = beat[:not_in[i]]
                if imsi_beat.to_numpy().shape[0] < 200:
                    continue
                while(np.array_equal(imsi_beat.to_numpy()[-1],np.array([0,0,0,0,0,0,0]))):
                    if len(imsi_beat) < 200:
                        break
                    else:
                        imsi_beat = imsi_beat[:-1]
                if len(imsi_beat) >= 200:
                    imsi_beat.to_csv('./milisec_200/'+filestr+'.csv.gzip', index = False, compression='gzip')
                    if 'beat' in midi.lower():
                        beat_count += 1
                    else:
                        fill_count += 1
            
            elif i == len(not_in):
                imsi_beat = beat[not_in[i-1]+1:]
                if imsi_beat.to_numpy().shape[0] < 200:
                    continue
                while(np.array_equal(imsi_beat.to_numpy()[-1],np.array([0,0,0,0,0,0,0]))):
                    if len(imsi_beat) < 200:
                        break
                    else:
                        imsi_beat = imsi_beat[:-1]
                if len(imsi_beat) >= 200:
                    print(imsi_beat.to_numpy()[-1])
                    imsi_beat.to_csv('./milisec_200/'+filestr+'.csv.gzip', index = False, compression='gzip')
                    if 'beat' in midi.lower():
                        beat_count += 1
                    else:
                        fill_count += 1
            else:
                imsi_beat = beat[not_in[i-1]+1:not_in[i]]
                if len(imsi_beat) > 0:
                    while(np.array_equal(imsi_beat.to_numpy()[-1],np.array([0,0,0,0,0,0,0]))):
                        if len(imsi_beat) < 200:
                            break
                        else:
                            imsi_beat = imsi_beat[:-1]
                    if len(imsi_beat) >= 200:
                        imsi_beat.to_csv('./milisec_200/'+filestr+'.csv.gzip', index = False, compression='gzip')
                        if 'beat' in midi.lower():
                            beat_count += 1
                        else:
                            fill_count += 1
    else:
        while(np.array_equal(beat.to_numpy()[-1], np.array([0,0,0,0,0,0,0]))):
            if len(beat) < 200:
                break
            else:
                beat = beat[:-1]
        if len(beat) >= 200:
            if 'beat' in midi.lower():
                filestr = 'beat' + str(beat_count)
            elif 'fill' in midi.lower():
                filestr = 'fill' + str(fill_count)
            beat.to_csv('./milisec_200/'+filestr+'.csv.gzip', index = False, compression='gzip')
            if 'beat' in midi.lower():
                beat_count += 1
            else:
                fill_count += 1

    
    # for i in range(len(beat)):
    #     softmax_beat.append(seven_kind_arr.index(beat.to_numpy()[i].tolist()))

    # if beat.shape[1] < 32:
    #     for i in range(beat.shape[1], 32):
    #         beat[i] = 0

    # print(softmax_beat)
    # print('----')
    # if count > 1:
    #     break
    # if len(beat) > 140:
    #     beat.to_csv('./milisec_dataset'+midi[27:]+'.csv.gzip', index = False, compression='gzip')
    # if len(beat) < min:
    #     min = len(beat)
    #     if len(beat) < 150:
    #         print(midi)
    #         print(beat.to_string())
    # if len(beat) > max:
    #     max = len(beat)

[0 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 0 0 0 0 0 1]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 0 0 1]
[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 0 1 0]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 0 1 0]
[1 1 0 0 0 0 0]
[1 0 0 0 0 1 0]
[0 1 0 0 0 0 0]
[1 1 0 0 0 0 0]
[1 1 0 0 0 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 0]
[0 0 0 0 0 1 0]
[1 0 0 0 0 0 0]
[0 0 0 0 0 1 0]
[1 0 0 0 0 0 0]
[1 1 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 0 1 0]
[0 1 0 0 0 0 0]
[0 1 0 0 1 0 0]
[1 0 0 0 0 0 0]
[0 1 0 0 1 0 0]
[0 1 0 0 0 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 1 0]
[0 0 0 1 0 0 0]
[0 0 0 0 0 1 0]
[0 0 0 1 0 0 0]
[1 0 0 0 0 0 1]
[0 0 1 0 0 0 0]
[1 0 0 0 0 0 1]
[0 0 0 0 0 1 0]
[0 0 0 0 0 0 1]
[0 0 0 0 1 0 0]
[0 0 0 0 0 0 1]
[0 0 0 0 0 1 0]
[0 1 0 0 0 0 0]
[1 0 0 0 0 0 1]
[1 0 0 0 0 0 1]
[1 0 0 0

In [3]:
csv_list = glob('./milisec_200/*.csv.gzip')

In [ ]:
for csv in csv_ist:
    imsi = pd.read_csv(csv, compression='gzip')
    if 'beat' in csv.lower():
        for _ in range(round(imsi.shape[1] // 140) * 1.2 ):
            a = random.randrange(0, imsi.shape[1] - 140)

In [7]:
from queue import Empty


fill_cnt = 1
beat_cnt = 1

cnt = 1

for csv in csv_list:
    imsi = pd.read_csv(csv, compression='gzip')
    sfmx = []
    for i in range(imsi.shape[0]):
        # print(imsi.to_numpy().tolist()[i])
        sfmx.append(seven_kind_arr.index(imsi.to_numpy().tolist()[i]))
    idx = np.where(21 != np.array(sfmx))[0]
    idx = idx[idx >= 199]
    # print(idx)
    print(csv)
    print(idx)
    if idx is not Empty:
        for _ in range(idx.max()//400 + 1):
            j = np.random.choice(idx, 1)
            picked = np.array(sfmx)[int(j)-199:int(j)+1]
            
            onehot = np.zeros((200,22))
            for i in range(200):
                onehot[i][picked[i]] = 1
            
            pd.DataFrame(onehot).to_csv('./milisec_200_dataset/'+str(cnt)+'.csv.gzip', compression='gzip')
            cnt += 1
    print(cnt)
        

    
    

    # if 'beat' in csv.lower():
    #     if imsi.shape[1] > 140:
    #         for i in range(round((imsi.shape[1] // 140) * 1.2 )):
    #             a = random.randrange(200, imsi.shape[1])
    #             two_bars = imsi.iloc[:,a-140:a]
    #             two_bars.to_csv('./dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
    #             two_bars[random.choices(two_bars.columns, k=3)] = 0
    #             two_bars.to_csv('./masked_dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
    #             beat_cnt += 1
    # elif 'fill' in csv.lower():
    #     if imsi.shape[1] == 32:
    #         imsi.to_csv('./dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
    #         imsi[random.choices(imsi.columns, k=3)] = 0
    #         imsi.to_csv('./masked_dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
    #         fill_cnt += 1
    #     else:
    #         for i in range(imsi.shape[1] // 16 // 2):
    #             a = random.randrange(0, imsi.shape[1]-32)
    #             two_bars = imsi.iloc[:,a:a+32]
    #             two_bars.to_csv('./dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
    #             two_bars[random.choices(two_bars.columns, k=3)] = 0
    #             two_bars.to_csv('./masked_dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
    #             fill_cnt += 1
    

        



./milisec_200\beat1.csv.gzip
[ 205  217  228  240  253  264  276  278  288  299  311  322  335  336
  348  360  371  384  396  408  420  432  445  456  468  480  492  493
  504  515  528  552  564  576  588  591  595  600  612  625  648  671
  683  694  706  718  721  742  766  767  779  791  802  804  816  828
  829  840  852  864  876  888  900  913  926  937  949  961  962  974
  985  997 1009 1021 1033 1044 1056 1079 1090 1103 1104 1116 1128 1140
 1152 1165 1176 1188 1200 1213 1214 1224 1237 1248 1261 1273 1285 1297
 1310 1321 1333 1344 1356 1361 1393 1415 1427 1438 1451 1462 1463 1477
 1486 1510 1521 1534 1558 1570 1571 1583 1595 1607 1620 1631 1644 1656
 1668 1680 1692 1704 1716 1728 1729 1741 1753 1764 1765 1777 1789 1801
 1812 1824 1835 1848 1859 1872 1895 1907 1919 1942 1955 1967 1979 1990
 2002 2015 2027 2028 2040 2052 2064 2076 2088 2100 2113 2124 2128 2134
 2146 2158 2166 2173 2182 2190 2198 2206 2207 2220 2226 2231 2245 2254
 2262 2270 2278 2287 2294 2302 2305 2328 2339 23

In [129]:
np.zeros((3,2))

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [36]:
a = pd.read_csv('milisec_dataset/drummer1/session1/6_jazz-funk_116_fill_4-4.mid.csv.gzip', compression='gzip')

In [ ]:
print(a.to_string())

In [59]:
seven_kind_arr.index(a.to_numpy()[0].tolist())

1

In [112]:
arr = np.array([1,2,3,1,2,1,3,1,1,2,3])

ll = [1,3]

k = []

for i in range(len(arr)):
    if arr[i] not in ll:
        k.append(i)


In [67]:
k

[1, 4, 9]

In [76]:
for i in range(len(k)+1):
    print(i)
    # print(k[i])
    if i == 0:
        print('0000')
        print(arr[:k[i]])
    elif i == len(k):
        print('fin')
        print(arr[k[i-1]+1:])
    else:
        print('else')
        print(arr[k[i-1]+1:k[i]])

0
0000
[1]
1
else
[3 1]
2
else
[1 3 1 1]
3
fin
[3]


In [50]:
np.delete(arr2, k)

IndexError: index 9 is out of bounds for axis 0 with size 6

In [28]:
seven_kind_arr.index([0,0,0,0,0,0,0])

21

In [9]:
asdf = np.array([[1,2,3],[4,5,6]])

AttributeError: 'numpy.ndarray' object has no attribute 'pop'